In [1]:
# required libraries
from mesa.batchrunner import BatchRunner
from model import MemeModel, number_susceptible, number_peak_meme_A, number_peak_meme_B
from model import step_peak_meme_A, step_peak_meme_B, number_bored_A, number_bored_B
from model import number_bored_both, number_interest_A, number_interest_B, number_interest_both
from model import number_steps

In [2]:
# running experiment using mesa batchrunner
# 1. using default params, running through batches
fixed_params = {
    "initial_viral_size_A": 5,
    "initial_viral_size_B": 5,
    "meme_spread_chance": 0.3,
    "maybe_bored": 0.3,
    "influencer_spread_chance": 0.6,
    "interest_meme_A_chance": 0.5,
    "interest_meme_B_chance": 0.5
}

variable_params = {
    "num_nodes": [100, 200, 300, 400, 500],
    "n_groups": [2, 3, 5, 7],
    "influencer_appearance": [1, 3, 5, 7]
}

batch_run = BatchRunner(
    MemeModel,
    variable_parameters=variable_params,
    fixed_parameters=fixed_params,
    iterations=10,
    max_steps=100,
    model_reporters={
        "susceptible": number_susceptible,
        "num_steps": number_steps,
        "peak_A": number_peak_meme_A,
        "step_peak_A": step_peak_meme_A,
        "peak_B": number_peak_meme_B,
        "step_peak_B": step_peak_meme_B,
        "bored_A": number_bored_A,
        "bored_B": number_bored_B,
        "bored_both": number_bored_both,
        "interest_A": number_interest_A,
        "interest_B": number_interest_B,
        "interest_both": number_interest_both
    }
)

batch_run.run_all()

batch_run_results = batch_run.get_model_vars_dataframe()
batch_run_results.head()

800it [02:39,  5.02it/s]


,num_nodes,n_groups,influencer_appearance,Run,Bored_A,Bored_B,Bored_both,Interest_A,Interest_B,Interest_both,...,Step_peak_A,Step_peak_B,Susceptible,initial_viral_size_A,initial_viral_size_B,meme_spread_chance,maybe_bored,influencer_spread_chance,interest_meme_A_chance,interest_meme_B_chance
0,100,2,1,0,37,18,3,56,43,26,...,7,5,48,5,5,0.3,0.3,0.6,0.5,0.5
1,100,2,1,1,26,41,10,48,49,21,...,2,4,43,5,5,0.3,0.3,0.6,0.5,0.5
2,100,2,1,2,38,22,4,51,54,34,...,4,3,44,5,5,0.3,0.3,0.6,0.5,0.5
3,100,2,1,3,74,7,1,55,63,33,...,4,2,20,5,5,0.3,0.3,0.6,0.5,0.5
4,100,2,1,4,45,40,19,52,53,29,...,5,6,34,5,5,0.3,0.3,0.6,0.5,0.5


In [3]:
# generate csv file from the batch run
batch_run_results.to_csv('batch_result_1.tsv', sep='\t', index=False)

In [4]:
# running experiment using mesa batchrunner
# 2. more viral size for Meme A but more interest for Meme B

fixed_params = {
    "initial_viral_size_A": 10,
    "initial_viral_size_B": 5,
    "meme_spread_chance": 0.3,
    "maybe_bored": 0.3,
    "influencer_spread_chance": 0.6,
    "interest_meme_A_chance": 0.3,
    "interest_meme_B_chance": 0.6
}

variable_params = {
    "num_nodes": [100, 200, 300, 400, 500],
    "n_groups": [2, 3, 5, 7],
    "influencer_appearance": [1, 3, 5, 7]
}

batch_run_2 = BatchRunner(
    MemeModel,
    variable_parameters=variable_params,
    fixed_parameters=fixed_params,
    iterations=10,
    max_steps=100,
    model_reporters={
        "susceptible": number_susceptible,
        "num_steps": number_steps,
        "peak_A": number_peak_meme_A,
        "step_peak_A": step_peak_meme_A,
        "peak_B": number_peak_meme_B,
        "step_peak_B": step_peak_meme_B,
        "bored_A": number_bored_A,
        "bored_B": number_bored_B,
        "bored_both": number_bored_both,
        "interest_A": number_interest_A,
        "interest_B": number_interest_B,
        "interest_both": number_interest_both
    }
)

batch_run_2.run_all()

batch_run_2_results = batch_run_2.get_model_vars_dataframe()
batch_run_2_results.head()

800it [02:44,  4.86it/s]


,num_nodes,n_groups,influencer_appearance,Run,bored_A,bored_B,bored_both,interest_A,interest_B,interest_both,...,step_peak_A,step_peak_B,susceptible,initial_viral_size_A,initial_viral_size_B,meme_spread_chance,maybe_bored,influencer_spread_chance,interest_meme_A_chance,interest_meme_B_chance
0,100,2,1,0,39,69,25,39,67,30,...,2,4,17,10,5,0.3,0.3,0.6,0.3,0.6
1,100,2,1,1,42,33,9,26,57,13,...,3,15,34,10,5,0.3,0.3,0.6,0.3,0.6
2,100,2,1,2,53,61,24,39,66,28,...,4,4,10,10,5,0.3,0.3,0.6,0.3,0.6
3,100,2,1,3,23,34,11,45,57,27,...,1,5,54,10,5,0.3,0.3,0.6,0.3,0.6
4,100,2,1,4,26,28,9,36,56,20,...,2,3,55,10,5,0.3,0.3,0.6,0.3,0.6


In [5]:
# generate csv file from the batch run
batch_run_2_results.to_csv('batch_result_2.tsv', sep='\t', index=False)